In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Bidirectional, CuDNNLSTM, LeakyReLU, BatchNormalization, Reshape, Input
from keras.callbacks import ModelCheckpoint
import os
from IPython.display import HTML, display

In [4]:
print(os.listdir('/content/gdrive/My Drive/midi_files'))

['.ipynb_checkpoints', '1_funk-groove1_138_beat_4-4.mid', '2_funk-groove2_105_beat_4-4.mid', '3_soul-groove3_86_beat_4-4.mid', '4_soul-groove4_80_beat_4-4.mid', '5_funk-groove5_84_beat_4-4.mid', '6_hiphop-groove6_87_beat_4-4.mid', '7_pop-groove7_138_beat_4-4.mid', '8_rock-groove8_65_beat_4-4.mid', '9_soul-groove9_105_beat_4-4.mid', '10_soul-groove10_102_beat_4-4.mid', '1_rock_105_beat_4-4.mid', '2_rock_105_beat_4-4.mid', '4_rock_105_beat_4-4.mid', '3_rock_105_beat_4-4.mid', '5_rock_105_beat_4-4.mid', '6_rock_105_beat_4-4.mid', '7_rock_105_beat_4-4.mid', '8_jazz_105_beat_4-4.mid', '9_rock_105_beat_4-4.mid', '10_rock_105_beat_4-4.mid', '11_rock_120_beat_4-4.mid', '12_rock_120_beat_4-4.mid', '13_rock_120_beat_4-4.mid', '14_rock_120_beat_4-4.mid', '16_jazz_120_beat_4-4.mid', '17_jazz_120_beat_4-4.mid', '15_jazz_120_beat_4-4.mid', '18_rock_120_beat_4-4.mid', '20_rock_120_beat_4-4.mid', '19_jazz_120_beat_4-4.mid', '21_jazz_120_beat_4-4.mid', '22_jazz_120_beat_4-4.mid', '23_jazz_120_beat_4-4.

In [5]:
def progress(file, value, max=100):
  return HTML("""
      <p>{file}</p>
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(file=file, value=value, max=max))

out = display(progress(0, 100), display_id=True)

In [6]:
def getNotes() :
  DIR = '/content/gdrive/My Drive/midi_files'
  num_files = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
  notes = []
  i = 1
  for file in glob.glob(DIR + '/*.mid'):
      midi = converter.parse(file)
      out.update(progress(file, i, num_files))
      i = i + 1

      notes_to_parse = None
      parts = instrument.partitionByInstrument(midi)

      if parts:
          notes_to_parse = parts.parts[0].recurse()
      else:
          notes_to_parse = midi.flat.notes

      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))
  out.update(progress('COMPLETE', num_files, num_files))
  with open('notes', 'wb') as filepath:
      pickle.dump(notes, filepath)
  return notes

In [7]:
notes = getNotes()
print(str(len(notes)) + ' Total Notes')
print(str(len(set(notes))) + ' Unique Notes')

10378 Total Notes
179 Unique Notes


In [8]:
sequence_length = 500
generator_input = 100
total_notes = len(notes)
unique_notes = len(set(notes))
pitchnames = sorted(set(notes))
print(pitchnames)
note_to_int = dict( (note, number) for number, note in enumerate(pitchnames))
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

['0', '0.1', '0.1.2', '0.1.3', '0.1.6', '0.2', '0.2.3', '0.2.4.7', '0.2.6', '0.2.7', '0.3', '0.3.7', '0.4', '0.4.6', '0.4.7', '0.4.8', '0.5', '0.6', '1.2', '1.3', '1.3.8', '1.5.8', '1.6', '1.7', '10.0', '10.0.1', '10.0.2', '10.0.4', '10.1', '10.11', '10.2', '10.2.4', '10.2.5', '10.3', '11', '11.0', '11.1', '11.2', '11.2.5.7', '11.3.6', '11.4', '2', '2.3', '2.3.8', '2.4', '2.5', '2.5.9', '2.6', '2.6.7.9', '2.6.8', '2.6.9', '2.7', '2.8', '3', '3.4', '3.4.8', '3.4.8.10', '3.5', '3.6.8', '3.7', '3.7.10', '3.7.9.10', '3.8', '3.9', '4', '4.10', '4.5', '4.6', '4.6.8', '4.7', '4.7.11', '4.7.9', '4.8', '4.8.10', '4.9', '5', '5.10', '5.7', '5.7.0', '5.7.10', '5.8', '5.8.0', '5.8.10', '5.9', '5.9.0', '5.9.11.0', '6', '6.10', '6.7', '6.7.11', '6.8', '6.8.0', '6.8.1', '6.9', '7', '7.0', '7.10', '7.10.2', '7.11', '7.11.0', '7.11.2', '7.8', '7.8.0', '7.9', '8.0', '8.0.1', '8.0.3', '8.1', '8.10', '8.10.0', '8.10.2', '8.9', '9', '9.0', '9.0.2', '9.0.4', '9.11', '9.11.2', '9.11.4', '9.2', 'A0', 'A2', 'A

In [9]:
trainx = []
trainy = []

In [10]:
for i in range( 0, total_notes - sequence_length ):
  x_sequence = notes[i : i + sequence_length]
  y_sequence = notes[i + sequence_length]
  trainx.append([ note_to_int[note] for note in x_sequence ])
  trainy.append(note_to_int[y_sequence])

n_sequences = len(trainx)
print('Number of sequences: ' + str(n_sequences))
print('Sequence 1: ' + str(trainx[0][0]))
print('Sequence 1 output: ' + str(trainy[0]))

Number of sequences: 9878
Sequence 1: 11
Sequence 1 output: 160


In [11]:
print(trainx[0][0:10])
trainx = np.reshape(trainx, (n_sequences, sequence_length, 1))
print('trainx shape: ' + str(trainx.shape))

[11, 10, 43, 170, 151, 145, 170, 151, 145, 170]
trainx shape: (9878, 500, 1)


In [12]:
trainy = np_utils.to_categorical(trainy)

latent_size = (trainx.shape[1], trainx.shape[2])

print('Index of category: ' + str(np.argmax(trainy[0])))
print(trainx[0][0:10])

Index of category: 160
[[ 11]
 [ 10]
 [ 43]
 [170]
 [151]
 [145]
 [170]
 [151]
 [145]
 [170]]


In [13]:
discriminator = Sequential()
discriminator.add(CuDNNLSTM(512, input_shape=latent_size, return_sequences=True))
discriminator.add(Bidirectional(CuDNNLSTM(512)))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(128))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(64))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.trainable = False
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

discriminator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm (CuDNNLSTM)      (None, 500, 512)          1054720   
                                                                 
 bidirectional (Bidirectiona  (None, 1024)             4202496   
 l)                                                              
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                        

In [14]:
generator = Sequential()
generator.add(Dense(256, input_dim=generator_input))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(np.prod((trainx.shape[1], trainx.shape[2])), activation='tanh'))
generator.add(Reshape(latent_size))
generator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_6 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [15]:
gan_input = Input(shape=(generator_input,))
fake_seq = generator(gan_input)

gan_output = discriminator(fake_seq)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_1 (Sequential)   (None, 500, 1)            1202420   
                                                                 
 sequential (Sequential)     (None, 1)                 5954561   
                                                                 
Total params: 7,156,981
Trainable params: 1,198,836
Non-trainable params: 5,958,145
_________________________________________________________________


In [16]:
batch_size = 124
x_batch = trainx[ np.random.randint(0, trainx.shape[0], size=batch_size), : ]


In [18]:
noise = np.random.uniform( 0, 1, size=[batch_size, 100] )
fake = generator.predict(noise)
fake = (fake)*float(unique_notes)
fake = np.absolute(fake.astype(int))

In [19]:
offset = 0
output_notes = []
output = []
for i in range(len(fake)):
  output.append(fake[0][i][0])

print(output)
for pattern in output:
    if pattern > 357:
      pattern = 357
    n = int_to_note[pattern]
    if ('.' in n) or n.isdigit():
        notes_in_chord = n.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.SnareDrum()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(n)
        new_note.offset = offset
        new_note.storedInstrument = instrument.SnareDrum()
        output_notes.append(new_note)    
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/content/gdrive/My Drive/output_midi/test_output11.mid')

[23, 21, 17, 12, 42, 8, 39, 17, 23, 35, 9, 10, 46, 49, 7, 19, 0, 23, 24, 8, 14, 19, 7, 23, 42, 5, 5, 3, 23, 9, 12, 29, 6, 20, 31, 29, 26, 29, 27, 20, 15, 15, 23, 45, 32, 30, 7, 32, 6, 5, 2, 2, 21, 17, 20, 16, 13, 6, 45, 22, 1, 42, 40, 3, 35, 20, 25, 44, 4, 53, 27, 1, 33, 6, 28, 31, 18, 6, 34, 1, 18, 18, 39, 4, 17, 50, 55, 7, 9, 17, 9, 4, 28, 32, 48, 23, 18, 3, 39, 9, 7, 15, 15, 48, 1, 25, 11, 35, 12, 12, 5, 35, 22, 40, 3, 10, 23, 10, 16, 3, 33, 20, 25, 9]


'/content/gdrive/My Drive/output_midi/test_output11.mid'

In [20]:
from google.colab import files
files.download('/content/gdrive/My Drive/output_midi/test_output11.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>